# x402 Facilitator Demo

Test the x402 facilitator for Optimism and Base networks (mainnet & testnet).

In [27]:
import requests
import json
from datetime import datetime

# Facilitator endpoint - Single function with path-based routing
# FACILITATOR_URL = "https://facilitator.fretchen.eu"
FACILITATOR_URL = "http://localhost:8080"

VERIFY_URL = f"{FACILITATOR_URL}/verify"
SETTLE_URL = f"{FACILITATOR_URL}/settle"
SUPPORTED_URL = f"{FACILITATOR_URL}/supported"


print("🚀 x402 Facilitator Endpoints (Single Function with Path Routing):")
print(f"  Base: {FACILITATOR_URL}")
print(f"  Verify: {VERIFY_URL}")
print(f"  Settle: {SETTLE_URL}")
print(f"  Supported: {SUPPORTED_URL}")

🚀 x402 Facilitator Endpoints (Single Function with Path Routing):
  Base: http://localhost:8080
  Verify: http://localhost:8080/verify
  Settle: http://localhost:8080/settle
  Supported: http://localhost:8080/supported


## Test /supported Endpoint

The `/supported` endpoint returns information about which networks and payment schemes the facilitator supports.

**x402 v2 Changes:**
- Uses `x402Facilitator` class with registered schemes
- Returns `kinds` array with `{x402Version, scheme, network}` 
- Does NOT include asset details (USDC addresses, etc.) in response
- Asset information must be known by the client
- Custom `extensions` array shows additional capabilities (e.g., whitelist)

**Single Function with Path Routing:**
- Single Scaleway Function deployment
- Path routing: `/verify`, `/settle`, `/supported`
- x402 v2 standard compliant


In [28]:
# Test the /supported endpoint
response = requests.get(SUPPORTED_URL)

print(f"Status Code: {response.status_code}")
supported = response.json()
print(json.dumps(supported, indent=2))

print(f"\n✅ Supported Networks:")
for kind in supported['kinds']:
    print(f"  - {kind['network']} ({kind['scheme']} scheme, x402 v{kind['x402Version']})")
    # Note: x402 v2 does not include asset details in getSupported() response
    # Asset information (USDC addresses, etc.) must be known by the client

# Show custom extensions if present
if 'extensions' in supported and supported['extensions']:
    print(f"\n🔧 Custom Extensions:")
    for ext in supported['extensions']:
        print(f"  - {ext.get('name', 'unknown')}: {ext.get('description', '')}")


Status Code: 200
{
  "kinds": [
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:10"
    },
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:11155420"
    },
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:8453"
    },
    {
      "x402Version": 2,
      "scheme": "exact",
      "network": "eip155:84532"
    }
  ],
  "extensions": [
    {
      "name": "recipient_whitelist",
      "description": "Payment recipients must be authorized through smart contract whitelist. Clients can verify authorization by calling isAuthorizedAgent(address) on the contracts below.",
      "contracts": {
        "eip155:10": [
          {
            "name": "GenImNFTv4",
            "address": "0x80f95d330417a4acEfEA415FE9eE28db7A0A1Cdb",
            "method": "isAuthorizedAgent(address)"
          },
          {
            "name": "LLMv1",
            "address": "0x833F39D6e67390324796f861990ce9B7cf9F5dE1",
    

## Create an EIP-712 Signed Authorization

Now we'll create a signed EIP-712 message that authorizes the transfer.

**Supported Networks:**
| Chain | Testnet | Mainnet |
|-------|---------|---------|
| Optimism | `eip155:11155420` | `eip155:10` |
| Base | `eip155:84532` | `eip155:8453` |

**To switch networks:**
1. Set `USE_BASE = True` for Base, `False` for Optimism
2. Set `USE_MAINNET = True` for mainnet (⚠️ REAL MONEY!)



In [29]:
from eth_account import Account
from eth_account.messages import encode_typed_data
from eth_keys import keys
from web3 import Web3
import os
from dotenv import load_dotenv
import secrets

# Load environment variables
load_dotenv()
private_key = os.getenv('TEST_WALLET_PRIVATE_KEY')
# private_key = os.getenv("NFT_WALLET_PRIVATE_KEY")

# Create account from private key
account = Account.from_key(private_key)
from_address = Web3.to_checksum_address(account.address)

pay_to_address = os.getenv('NFT_WALLET_PUBLIC_KEY')
pay_to_address = Web3.to_checksum_address(pay_to_address)

# pay_to_address = "0x073f26F0C3FC100e7b075C3DC3cDE0A777497D20"

print(f"Payer Address: {from_address}")
print(f"Recipient Address: {pay_to_address}")

# ═══════════════════════════════════════════════════════════════
# ⚠️ NETWORK SELECTION - Choose your network here
# ═══════════════════════════════════════════════════════════════

# Step 1: Choose the chain (Optimism or Base)
USE_BASE = True  # Set to True for Base, False for Optimism

# Step 2: Choose testnet or mainnet
USE_MAINNET = False  # Set to True for mainnet with REAL MONEY

# Network configuration
NETWORK_CONFIG = {
    # Optimism networks
    ("optimism", False): {
        "chain_id": 11155420,
        "usdc_address": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7",
        "usdc_name": "USDC",
        "network_name": "Optimism Sepolia (Testnet)",
        "caip2": "eip155:11155420",
    },
    ("optimism", True): {
        "chain_id": 10,
        "usdc_address": "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85",
        "usdc_name": "USD Coin",
        "network_name": "Optimism Mainnet",
        "caip2": "eip155:10",
    },
    # Base networks
    ("base", False): {
        "chain_id": 84532,
        "usdc_address": "0x036CbD53842c5426634e7929541eC2318f3dCF7e",
        "usdc_name": "USDC",
        "network_name": "Base Sepolia (Testnet)",
        "caip2": "eip155:84532",
    },
    ("base", True): {
        "chain_id": 8453,
        "usdc_address": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
        "usdc_name": "USD Coin",
        "network_name": "Base Mainnet",
        "caip2": "eip155:8453",
    },
}

# Get the selected network configuration
chain_key = ("base" if USE_BASE else "optimism", USE_MAINNET)
config = NETWORK_CONFIG[chain_key]

CHAIN_ID = config["chain_id"]
USDC_ADDRESS = config["usdc_address"]
USDC_NAME = config["usdc_name"]
NETWORK_NAME = config["network_name"]
CAIP2_NETWORK = config["caip2"]

if USE_MAINNET:
    print(f"\n🚨 WARNING: Using REAL MONEY on {NETWORK_NAME}!")
else:
    print(f"\n🧪 Using testnet: {NETWORK_NAME}")

print(f"Chain ID: {CHAIN_ID}")
print(f"USDC Address: {USDC_ADDRESS}")
print(f"USDC Name: {USDC_NAME}")
print(f"CAIP-2 Network: {CAIP2_NETWORK}")

Payer Address: 0x553179556FC2A39e535D65b921e01fA995E79101
Recipient Address: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

🧪 Using testnet: Base Sepolia (Testnet)
Chain ID: 84532
USDC Address: 0x036CbD53842c5426634e7929541eC2318f3dCF7e
USDC Name: USDC
CAIP-2 Network: eip155:84532


### Create EIP-712 Signature

Now we create an EIP-712 signed authorization for `transferWithAuthorization` (EIP-3009):

In [30]:
# Create EIP-712 Signature for TransferWithAuthorization (EIP-3009)
# ✅ Use eth_keys for USDC-compatible signatures (manual EIP-712 hash)
import time
from eth_hash.auto import keccak

# Payment amount: $0.01 USDC (6 decimals)
payment_amount = "10000"  # 0.01 USDC = 10000 (6 decimals)

# Generate a random nonce (32 bytes)
nonce = "0x" + secrets.token_hex(32)

# Validity window
valid_after = 0  # Valid immediately
valid_before = int(time.time()) + 3600  # Valid for 1 hour

print(f"📝 Creating EIP-712 Authorization:")
print(f"   Network: {NETWORK_NAME}")
print(f"   Amount: ${int(payment_amount)/1e6:.2f} USDC")
print(f"   From: {from_address}")
print(f"   To: {pay_to_address}")
print(f"   Nonce: {nonce[:20]}...")
print(f"   Valid until: {datetime.fromtimestamp(valid_before)}")

# EIP-712 typed data structure for TransferWithAuthorization
full_message = {
    "types": {
        "EIP712Domain": [
            {"name": "name", "type": "string"},
            {"name": "version", "type": "string"},
            {"name": "chainId", "type": "uint256"},
            {"name": "verifyingContract", "type": "address"}
        ],
        "TransferWithAuthorization": [
            {"name": "from", "type": "address"},
            {"name": "to", "type": "address"},
            {"name": "value", "type": "uint256"},
            {"name": "validAfter", "type": "uint256"},
            {"name": "validBefore", "type": "uint256"},
            {"name": "nonce", "type": "bytes32"}
        ]
    },
    "primaryType": "TransferWithAuthorization",
    "domain": {
        "name": USDC_NAME,
        "version": "2",
        "chainId": CHAIN_ID,
        "verifyingContract": USDC_ADDRESS
    },
    "message": {
        "from": from_address,
        "to": pay_to_address,
        "value": int(payment_amount),
        "validAfter": valid_after,
        "validBefore": valid_before,
        "nonce": nonce
    }
}

# Encode to get the EIP-712 hash components
signable_message = encode_typed_data(full_message=full_message)

# Compute the FULL EIP-712 hash according to spec:
# keccak256("\x19\x01" || domainSeparator || hashStruct(message))
prefix = bytes.fromhex("1901")
domain_separator = signable_message.header  # This is hashStruct(domain)
message_hash = signable_message.body  # This is hashStruct(message)
full_eip712_hash = keccak(prefix + domain_separator + message_hash)

# Sign using eth_keys (this creates a pure EIP-712 signature)
private_key_bytes = bytes.fromhex(private_key.replace('0x', ''))
pk = keys.PrivateKey(private_key_bytes)
signature_obj = pk.sign_msg_hash(full_eip712_hash)

# Extract v, r, s components
v_raw, r, s = signature_obj.vrs

# Adjust v to be 27 or 28 (Ethereum standard)
if v_raw == 0:
    v = 27
elif v_raw == 1:
    v = 28
else:
    v = v_raw

# Combine into signature hex string: 0x + r + s + v
r_bytes = r.to_bytes(32, byteorder='big')
s_bytes = s.to_bytes(32, byteorder='big')
signature_hex = '0x' + r_bytes.hex() + s_bytes.hex() + format(v, '02x')

print(f"\n🔐 Signature Details:")
print(f"EIP-712 Hash: {full_eip712_hash.hex()}")
print(f"Signature: {signature_hex}")
print(f"Signature length: {len(signature_hex)} chars (should be 132 with 0x)")
print(f"v: {v}, r: {r_bytes.hex()[:16]}..., s: {s_bytes.hex()[:16]}...")
print(f"Nonce: {nonce}")
print(f"\n✅ Authorized ${int(payment_amount)/1e6:.2f} USDC payment")
print(f"\n💡 Using correct domain name: '{USDC_NAME}' for {NETWORK_NAME}")

📝 Creating EIP-712 Authorization:
   Network: Base Sepolia (Testnet)
   Amount: $0.01 USDC
   From: 0x553179556FC2A39e535D65b921e01fA995E79101
   To: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
   Nonce: 0x784b0e4eebe37f29c9...
   Valid until: 2026-01-27 18:31:24

🔐 Signature Details:
EIP-712 Hash: 1b9550f0ec9fe168bfe3941ca9a9bb55be6610759654ea051d5fb30cbc263777
Signature: 0x8fc915a3a31342cd8712bd23f3f597dc511b6ce43ec7cf1ca5dfaea3fad5935e7b9b20426dc7764e516cf61c66fe8e79741ae9c1bef9f5ed1af61d2cd1399a3a1c
Signature length: 132 chars (should be 132 with 0x)
v: 28, r: 8fc915a3a31342cd..., s: 7b9b20426dc7764e...
Nonce: 0x784b0e4eebe37f29c9bfb4db913498a1eb5d1832b5bc0a6d9eb3882ef4555c16

✅ Authorized $0.01 USDC payment

💡 Using correct domain name: 'USDC' for Base Sepolia (Testnet)


### Debug: Vergleiche EIP-712 Domain-Parameter

Prüfe, ob die Domain-Parameter (name, version) vom Contract mit unseren Annahmen übereinstimmen:

In [31]:
# Debug: Read actual EIP-712 domain parameters from the USDC contract
print(f"🔍 Debugging EIP-712 Domain Parameters for {NETWORK_NAME}")
print(f"Contract: {USDC_ADDRESS}\n")

# Define domain_data based on selected network configuration
domain_data = {
    "name": USDC_NAME,
    "version": "2",
    "chainId": CHAIN_ID,
    "verifyingContract": USDC_ADDRESS
}

# Try different methods to get domain info
methods_to_try = [
    # Method 1: ERC-5267 standard (eip712Domain)
    {
        "name": "eip712Domain",
        "abi": [{
            "inputs": [],
            "name": "eip712Domain",
            "outputs": [
                {"name": "fields", "type": "bytes1"},
                {"name": "name", "type": "string"},
                {"name": "version", "type": "string"},
                {"name": "chainId", "type": "uint256"},
                {"name": "verifyingContract", "type": "address"},
                {"name": "salt", "type": "bytes32"},
                {"name": "extensions", "type": "uint256[]"}
            ],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    # Method 2: Simple name() and version() getters
    {
        "name": "name",
        "abi": [{
            "inputs": [],
            "name": "name",
            "outputs": [{"name": "", "type": "string"}],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    {
        "name": "version",
        "abi": [{
            "inputs": [],
            "name": "version",
            "outputs": [{"name": "", "type": "string"}],
            "stateMutability": "view",
            "type": "function"
        }]
    },
    # Method 3: DOMAIN_SEPARATOR (computed hash)
    {
        "name": "DOMAIN_SEPARATOR",
        "abi": [{
            "inputs": [],
            "name": "DOMAIN_SEPARATOR",
            "outputs": [{"name": "", "type": "bytes32"}],
            "stateMutability": "view",
            "type": "function"
        }]
    }
]

# Connect to network based on selection
network_rpc = {
    "eip155:10": "https://mainnet.optimism.io",
    "eip155:11155420": "https://sepolia.optimism.io",
    "eip155:8453": "https://mainnet.base.org",
    "eip155:84532": "https://sepolia.base.org",
}

rpc_url = network_rpc.get(CAIP2_NETWORK, "https://sepolia.optimism.io")
w3 = Web3(Web3.HTTPProvider(rpc_url))

contract_info = {}

for method in methods_to_try:
    try:
        contract = w3.eth.contract(address=USDC_ADDRESS, abi=method["abi"])
        func = getattr(contract.functions, method["name"])
        result = func().call()
        contract_info[method["name"]] = result
        print(f"✅ {method['name']}(): {result}")
    except Exception as e:
        print(f"❌ {method['name']}(): {str(e)[:80]}...")

# Compare with our assumptions
print(f"\n📊 Comparison:")
print(f"Our assumptions:")
print(f"  name: '{domain_data['name']}'")
print(f"  version: '{domain_data['version']}'")
print(f"  chainId: {domain_data['chainId']}")
print(f"  verifyingContract: {domain_data['verifyingContract']}")

if "eip712Domain" in contract_info:
    fields, name, version, chainId, verifyingContract, salt, extensions = contract_info["eip712Domain"]
    print(f"\nContract's actual EIP-712 domain:")
    print(f"  name: '{name}'")
    print(f"  version: '{version}'")
    print(f"  chainId: {chainId}")
    print(f"  verifyingContract: {verifyingContract}")
    
    # Check for mismatches
    if name != domain_data['name']:
        print(f"\n⚠️  MISMATCH: name is '{name}' not '{domain_data['name']}'!")
    if version != domain_data['version']:
        print(f"⚠️  MISMATCH: version is '{version}' not '{domain_data['version']}'!")
    if chainId != domain_data['chainId']:
        print(f"⚠️  MISMATCH: chainId is {chainId} not {domain_data['chainId']}!")
    else:
        print(f"\n✅ All domain parameters match!")
else:
    if "name" in contract_info:
        print(f"\nContract's name: '{contract_info['name']}'")
        if contract_info['name'] != domain_data['name']:
            print(f"⚠️  MISMATCH: Expected '{domain_data['name']}'")
    
    if "version" in contract_info:
        print(f"Contract's version: '{contract_info['version']}'")
        if contract_info['version'] != domain_data['version']:
            print(f"⚠️  MISMATCH: Expected '{domain_data['version']}'")

if "DOMAIN_SEPARATOR" in contract_info:
    print(f"\n🔐 Contract's DOMAIN_SEPARATOR:")
    print(f"  {contract_info['DOMAIN_SEPARATOR'].hex()}")

🔍 Debugging EIP-712 Domain Parameters for Base Sepolia (Testnet)
Contract: 0x036CbD53842c5426634e7929541eC2318f3dCF7e

❌ eip712Domain(): ('execution reverted', '0x')...
✅ name(): USDC
✅ version(): 2
✅ DOMAIN_SEPARATOR(): b'q\xf1z;/\xf3s\xb8\x03\xd7\nZ\x07\xc0F\xc1\xa2\xbc\x8e\x89\xc0\x9e\xf7"\xfc\xb0G\xab\xe9L\x98\x18'

📊 Comparison:
Our assumptions:
  name: 'USDC'
  version: '2'
  chainId: 84532
  verifyingContract: 0x036CbD53842c5426634e7929541eC2318f3dCF7e

Contract's name: 'USDC'
Contract's version: '2'

🔐 Contract's DOMAIN_SEPARATOR:
  71f17a3b2ff373b803d70a5a07c046c1a2bc8e89c09ef722fcb047abe94c9818


In [32]:
# Check if the current nonce was already used on-chain
print(f"Prüfe Nonce: {nonce}")
print(f"Payer: {from_address}")
print(f"Network: {NETWORK_NAME}")

# USDC contract ABI for authorizationState
usdc_abi = [
    {
        "inputs": [
            {"name": "authorizer", "type": "address"},
            {"name": "nonce", "type": "bytes32"}
        ],
        "name": "authorizationState",
        "outputs": [{"name": "", "type": "bool"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Connect to appropriate network
if USE_MAINNET:
    rpc_url = 'https://mainnet.optimism.io'
else:
    rpc_url = 'https://sepolia.optimism.io'

w3 = Web3(Web3.HTTPProvider(rpc_url))
usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=usdc_abi)

# Check if nonce is used
nonce_used = usdc_contract.functions.authorizationState(from_address, nonce).call()

if nonce_used:
    print(f"\n❌ Diese Nonce wurde bereits verwendet!")
    print(f"Du musst die vorherige Zelle erneut ausführen, um eine neue Nonce zu generieren.")
else:
    print(f"\n✅ Diese Nonce ist noch verfügbar und kann verwendet werden.")


Prüfe Nonce: 0x784b0e4eebe37f29c9bfb4db913498a1eb5d1832b5bc0a6d9eb3882ef4555c16
Payer: 0x553179556FC2A39e535D65b921e01fA995E79101
Network: Base Sepolia (Testnet)

✅ Diese Nonce ist noch verfügbar und kann verwendet werden.


### 🔐 Test: Validiere Signatur lokal

Bevor wir zum Server senden, testen wir ob unsere Signatur überhaupt valide ist:

In [33]:
# Now create a valid x402 payment payload with our real signature

real_payment_payload = {
    "x402Version": 2,
    "resource": {
        "url": "https://api.example.com/premium-data",
        "description": "Access to premium market data",
        "mimeType": "application/json"
    },
    "accepted": {
        "scheme": "exact",
        "network": CAIP2_NETWORK,  # Dynamic: eip155:10 (mainnet) or eip155:11155420 (testnet)
        "amount": payment_amount,
        "asset": USDC_ADDRESS,
        "payTo": pay_to_address,
        "maxTimeoutSeconds": 60,
        "extra": {
            "name": USDC_NAME,  # CRITICAL: Must match what we signed with!
            "version": "2"
        }
    },
    "payload": {
        "signature": signature_hex,
        "authorization": {
            "from": from_address,
            "to": pay_to_address,
            "value": int(payment_amount),
            "validAfter": int(valid_after),
            "validBefore": int(valid_before),
            "nonce": nonce
        }
    }
}

# Create payment requirements
real_payment_requirements = {
    "scheme": "exact",
    "network": CAIP2_NETWORK,  # Dynamic network
    "amount": payment_amount,
    "asset": USDC_ADDRESS,
    "payTo": pay_to_address,
    "maxTimeoutSeconds": 60,
    "extra": {
        "name": USDC_NAME,  # CRITICAL: Must match what we signed with!
        "version": "2"
    }
}

# Create verify request
real_verify_request = {
    "paymentPayload": real_payment_payload,
    "paymentRequirements": real_payment_requirements
}

print(f"\n📤 Request Summary:")
print(f"  Network: {NETWORK_NAME}")
print(f"  Payment Amount: ${int(payment_amount)/1e6:.2f} USDC")
print(f"  Recipient: {pay_to_address}")

# Send to facilitator (deployed on Scaleway)
response = requests.post(
    VERIFY_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request
)

result = response.json()
print(f"\n✅ Verify Response (Status {response.status_code}):")
print(json.dumps(result, indent=2))

if result.get('isValid'):
    print("\n🎉 Payment signature is valid and ready for settlement!")
    print(f"   Facilitator will transfer ${int(payment_amount)/1e6:.2f} to recipient")
else:
    invalid_reason = result.get('invalidReason', 'unknown')
    print(f"\n❌ Validation failed: {invalid_reason}")
    
    # If insufficient funds, check actual balance
    if 'insufficient' in invalid_reason.lower():
        print(f"\n💰 Checking payer's USDC balance...")
        
        # USDC ERC-20 ABI for balanceOf
        erc20_abi = [
            {
                "inputs": [{"name": "account", "type": "address"}],
                "name": "balanceOf",
                "outputs": [{"name": "", "type": "uint256"}],
                "stateMutability": "view",
                "type": "function"
            }
        ]
        
        # Connect to network
        if USE_MAINNET:
            rpc_url = 'https://mainnet.optimism.io'
        else:
            rpc_url = 'https://sepolia.optimism.io'
        
        w3 = Web3(Web3.HTTPProvider(rpc_url))
        usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=erc20_abi)
        
        # Check balance
        try:
            payer_balance = usdc_contract.functions.balanceOf(from_address).call()
            required_amount = int(payment_amount)
            
            print(f"\n📊 Balance Check:")
            print(f"   Payer address: {from_address}")
            print(f"   Current USDC balance: ${payer_balance/1e6:.6f} USDC")
            print(f"   Required for payment: ${required_amount/1e6:.6f} USDC")
            print(f"   Shortfall: ${(required_amount - payer_balance)/1e6:.6f} USDC")
            
            if payer_balance == 0:
                print(f"\n💡 Payer has NO USDC! Get USDC:")
                if USE_MAINNET:
                    print(f"   • Bridge: https://app.optimism.io/bridge")
                    print(f"   • Buy: https://www.coinbase.com/price/usd-coin")
                else:
                    print(f"   • Faucet: https://faucet.circle.com/")
                    print(f"   • Or bridge from Sepolia: https://app.optimism.io/bridge")
            else:
                print(f"\n💡 Payer needs more USDC:")
                print(f"   Send at least ${(required_amount - payer_balance)/1e6:.6f} USDC to:")
                print(f"   {from_address}")
                
        except Exception as e:
            print(f"\n⚠️  Could not check balance: {str(e)}")



📤 Request Summary:
  Network: Base Sepolia (Testnet)
  Payment Amount: $0.01 USDC
  Recipient: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

✅ Verify Response (Status 200):
{
  "isValid": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101"
}

🎉 Payment signature is valid and ready for settlement!
   Facilitator will transfer $0.01 to recipient


## Settlement via Facilitator

Nach erfolgreicher Verifikation kann der Facilitator die Zahlung on-chain ausführen. Der `/settle` Endpoint:
1. Führt nochmals alle Verifikationen durch
2. Ruft `transferWithAuthorization` auf dem USDC-Contract auf
3. Gibt die Transaction Hash zurück

**Voraussetzungen für Settlement:**
- Der Payer braucht USDC auf Optimism Sepolia
- Der Facilitator braucht einen Private Key (`FACILITATOR_WALLET_PRIVATE_KEY` in `.env`) und etwas ETH für Gas
- Für diesen Test kannst du deinen eigenen Private Key verwenden (im echten Betrieb sollte der Facilitator einen separaten Wallet haben)

### Prüfe Facilitator-Wallet Balance

Bevor wir Settlement versuchen, prüfen wir ob der Facilitator genug ETH für Gas hat:

In [34]:
# Check facilitator wallet balance AND payer USDC balance
# Now supports all 4 networks (Optimism + Base, Mainnet + Testnet)

facilitator_address = os.getenv('FACILITATOR_WALLET_PUBLIC_KEY')

# RPC endpoints for all networks
network_rpc = {
    "eip155:10": "https://mainnet.optimism.io",
    "eip155:11155420": "https://sepolia.optimism.io",
    "eip155:8453": "https://mainnet.base.org",
    "eip155:84532": "https://sepolia.base.org",
}

# Faucet/bridge URLs
network_faucets = {
    "eip155:10": "Bridge: https://app.optimism.io/bridge",
    "eip155:11155420": "Faucet: https://app.optimism.io/faucet",
    "eip155:8453": "Bridge: https://bridge.base.org",
    "eip155:84532": "Faucet: https://www.alchemy.com/faucets/base-sepolia",
}

if facilitator_address:
    facilitator_address = Web3.to_checksum_address(facilitator_address)
    
    # Connect to the SELECTED network (based on USE_BASE and USE_MAINNET)
    rpc_url = network_rpc.get(CAIP2_NETWORK, "https://sepolia.optimism.io")
    w3 = Web3(Web3.HTTPProvider(rpc_url))
    
    if w3.is_connected():
        print(f"🔍 Pre-Settlement Check for {NETWORK_NAME}")
        print(f"   Network ID: {CAIP2_NETWORK}")
        print("=" * 60)
        
        # 1. Check Facilitator ETH Balance (for gas)
        balance_wei = w3.eth.get_balance(facilitator_address)
        balance_eth = w3.from_wei(balance_wei, 'ether')
        
        print(f"\n💰 Facilitator Wallet (pays gas):")
        print(f"   Address: {facilitator_address}")
        print(f"   ETH Balance: {balance_eth} ETH")
        
        # Estimate gas cost (rough estimate: ~100k gas)
        estimated_gas = 100000
        gas_price = w3.eth.gas_price
        estimated_cost_wei = estimated_gas * gas_price
        estimated_cost_eth = w3.from_wei(estimated_cost_wei, 'ether')
        
        print(f"   Estimated Gas: ~{estimated_cost_eth:.6f} ETH")
        
        if balance_wei == 0:
            print(f"\n   ❌ ERROR: Facilitator has NO ETH for gas!")
            print(f"   Send ETH to: {facilitator_address}")
            print(f"   {network_faucets.get(CAIP2_NETWORK, 'Get ETH from faucet/bridge')}")
        elif balance_wei < estimated_cost_wei:
            print(f"\n   ⚠️  WARNING: ETH balance might be too low for gas")
            print(f"   Recommended: At least {estimated_cost_eth:.6f} ETH")
        else:
            print(f"   ✅ Sufficient ETH for gas")
        
        # 2. Check Payer USDC Balance (for payment)
        erc20_abi = [
            {
                "inputs": [{"name": "account", "type": "address"}],
                "name": "balanceOf",
                "outputs": [{"name": "", "type": "uint256"}],
                "stateMutability": "view",
                "type": "function"
            }
        ]
        
        usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=erc20_abi)
        payer_usdc_balance = usdc_contract.functions.balanceOf(from_address).call()
        required_amount = int(payment_amount)
        
        print(f"\n💵 Payer Wallet (sends USDC):")
        print(f"   Address: {from_address}")
        print(f"   USDC Balance: ${payer_usdc_balance/1e6:.6f} USDC")
        print(f"   Required: ${required_amount/1e6:.6f} USDC")
        
        if payer_usdc_balance == 0:
            print(f"\n   ❌ ERROR: Payer has NO USDC!")
            print(f"   Get USDC from: https://faucet.circle.com/ (select {NETWORK_NAME})")
        elif payer_usdc_balance < required_amount:
            shortfall = required_amount - payer_usdc_balance
            print(f"\n   ❌ ERROR: Insufficient USDC!")
            print(f"   Shortfall: ${shortfall/1e6:.6f} USDC")
            print(f"   Get USDC from: https://faucet.circle.com/")
        else:
            print(f"   ✅ Sufficient USDC for payment")
        
        # 3. Summary
        print(f"\n" + "=" * 60)
        facilitator_ok = balance_wei >= estimated_cost_wei
        payer_ok = payer_usdc_balance >= required_amount
        
        if facilitator_ok and payer_ok:
            print(f"✅ Ready for settlement on {NETWORK_NAME}")
        else:
            print(f"❌ Cannot settle - fix the issues above:")
            if not facilitator_ok:
                print(f"   • Facilitator needs ETH on {NETWORK_NAME}")
            if not payer_ok:
                print(f"   • Payer needs USDC on {NETWORK_NAME}")
    else:
        print(f"❌ Could not connect to {NETWORK_NAME}")
else:
    print("❌ FACILITATOR_WALLET_PUBLIC_KEY not configured in .env")

🔍 Pre-Settlement Check for Base Sepolia (Testnet)
   Network ID: eip155:84532

💰 Facilitator Wallet (pays gas):
   Address: 0x3F8d2Fb6fEA24E70155bC61471936F3c9C30c206
   ETH Balance: 0.01 ETH
   Estimated Gas: ~0.000000 ETH
   ✅ Sufficient ETH for gas

💵 Payer Wallet (sends USDC):
   Address: 0x553179556FC2A39e535D65b921e01fA995E79101
   USDC Balance: $19.990000 USDC
   Required: $0.010000 USDC
   ✅ Sufficient USDC for payment

✅ Ready for settlement on Base Sepolia (Testnet)


In [35]:
# Try to settle the payment (execute on-chain)
# Note: This will fail if the payer doesn't have USDC or if the facilitator wallet has no ETH for gas

print(f"\n💸 Attempting Settlement...")
print(f"  Network: {NETWORK_NAME}")
print(f"  Payment: ${int(payment_amount)/1e6:.2f} USDC → {pay_to_address}")

if USE_MAINNET:
    print(f"\n🚨 WARNING: This will execute a REAL transaction with REAL MONEY!")
    confirm = input("Type 'YES' to confirm: ")
    if confirm != 'YES':
        print("❌ Settlement cancelled")
        raise Exception("User cancelled mainnet settlement")

settle_response = requests.post(
    SETTLE_URL,
    headers={"Content-Type": "application/json"},
    json=real_verify_request  # Same request structure as verify
)

settle_result = settle_response.json()
print(f"\n📦 Settle Response (Status {settle_response.status_code}):")
print(json.dumps(settle_result, indent=2))

if settle_result.get('success'):
    tx_hash = settle_result.get('transaction')  # API returns 'transaction', not 'transactionHash'
    print(f"\n🎉 Settlement erfolgreich!")
    print(f"\n📝 Transaction Details:")
    print(f"   Hash: {tx_hash}")
    print(f"   Recipient received: ${int(payment_amount)/1e6:.2f} USDC")
else:
    tx_hash = None
    print(f"\n❌ Settlement fehlgeschlagen")
    error_reason = settle_result.get('errorReason', settle_result.get('invalidReason', 'unknown'))
    print(f"   Grund: {error_reason}")
    
    if 'insufficient' in error_reason.lower():
        print(f"\n💡 Tipp: Stelle sicher, dass:")
        print(f"   - Payer hat ${int(payment_amount)/1e6:.2f} USDC auf {NETWORK_NAME}")
        print(f"   - Facilitator hat ETH für Gas")



💸 Attempting Settlement...
  Network: Base Sepolia (Testnet)
  Payment: $0.01 USDC → 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C

📦 Settle Response (Status 200):
{
  "success": true,
  "payer": "0x553179556FC2A39e535D65b921e01fA995E79101",
  "transaction": "0x6379e34692892294bc8edc93159200b9232be913c2ab93fcd902e06fe3d25781",
  "network": "eip155:84532"
}

🎉 Settlement erfolgreich!

📝 Transaction Details:
   Hash: 0x6379e34692892294bc8edc93159200b9232be913c2ab93fcd902e06fe3d25781
   Recipient received: $0.01 USDC


In [36]:
# Display transaction on block explorer
# ✅ WICHTIG: Wir nutzen das 'network' Feld aus der Server-Response, nicht die lokale Variable!

if tx_hash:
    # Network aus der Response extrahieren (x402 Standard)
    response_network = settle_result.get('network', '')
    
    # Cross-Chain Validierung: Stimmt die Response mit unserem erwarteten Network überein?
    if response_network and response_network != CAIP2_NETWORK:
        print(f"⚠️  WARNUNG: Network-Mismatch!")
        print(f"   Erwartet: {CAIP2_NETWORK} ({NETWORK_NAME})")
        print(f"   Response: {response_network}")
        print(f"   Die Transaktion wurde möglicherweise auf einem anderen Network ausgeführt!")
    
    # Block Explorer URL aus der Response ableiten (nicht aus lokaler Variable!)
    network_to_explorer = {
        "eip155:10": "https://optimistic.etherscan.io/tx/",  # Optimism Mainnet
        "eip155:11155420": "https://sepolia-optimism.etherscan.io/tx/",  # Optimism Sepolia
        "eip155:8453": "https://basescan.org/tx/",  # Base Mainnet
        "eip155:84532": "https://sepolia.basescan.org/tx/",  # Base Sepolia
    }
    
    if response_network in network_to_explorer:
        explorer_url = network_to_explorer[response_network] + tx_hash
        network_display = response_network
    else:
        # Fallback auf lokale Variable (mit Warnung)
        print(f"⚠️  Unbekanntes Network '{response_network}', nutze lokale Konfiguration")
        if USE_MAINNET:
            explorer_url = f"https://optimistic.etherscan.io/tx/{tx_hash}"
        else:
            explorer_url = f"https://sepolia-optimism.etherscan.io/tx/{tx_hash}"
        network_display = CAIP2_NETWORK
    
    print(f"\n🔍 Block Explorer:")
    print(f"   {explorer_url}")
    print(f"\n📊 Transaktionsdetails:")
    print(f"   • Transaction hash: {tx_hash}")
    print(f"   • Network (Response): {response_network}")
    print(f"   • Network (Erwartet): {CAIP2_NETWORK}")
    print(f"   • Token: USDC at {USDC_ADDRESS}")
else:
    print(f"\n⚠️  No transaction hash available")


🔍 Block Explorer:
   https://sepolia.basescan.org/tx/0x6379e34692892294bc8edc93159200b9232be913c2ab93fcd902e06fe3d25781

📊 Transaktionsdetails:
   • Transaction hash: 0x6379e34692892294bc8edc93159200b9232be913c2ab93fcd902e06fe3d25781
   • Network (Response): eip155:84532
   • Network (Erwartet): eip155:84532
   • Token: USDC at 0x036CbD53842c5426634e7929541eC2318f3dCF7e


### 🔍 Cross-Chain Balance Verification

Nach dem Settlement prüfen wir die Balances auf **beiden** Networks, um sicherzustellen, dass die Transaktion auf dem richtigen Network ausgeführt wurde:

In [ ]:
# Cross-Chain Balance Verification
# Prüfe USDC Balances auf ALLEN Networks um sicherzustellen, dass Settlement korrekt war

from web3 import Web3

# USDC ERC-20 balanceOf ABI
balance_abi = [
    {
        "inputs": [{"name": "account", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "", "type": "uint256"}],
        "stateMutability": "view",
        "type": "function"
    }
]

# Network configurations - all 4 supported networks
networks = {
    "eip155:10": {
        "name": "Optimism Mainnet",
        "rpc": "https://mainnet.optimism.io",
        "usdc": "0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85"
    },
    "eip155:11155420": {
        "name": "Optimism Sepolia",
        "rpc": "https://sepolia.optimism.io", 
        "usdc": "0x5fd84259d66Cd46123540766Be93DFE6D43130D7"
    },
    "eip155:8453": {
        "name": "Base Mainnet",
        "rpc": "https://mainnet.base.org",
        "usdc": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913"
    },
    "eip155:84532": {
        "name": "Base Sepolia",
        "rpc": "https://sepolia.base.org",
        "usdc": "0x036CbD53842c5426634e7929541eC2318f3dCF7e"
    }
}

print(f"🔍 Cross-Chain Balance Check für Recipient: {pay_to_address}")
print(f"   Erwartetes Settlement-Network: {CAIP2_NETWORK}\n")

# Group by chain type for better readability
print("📊 Optimism Networks:")
for network_id in ["eip155:10", "eip155:11155420"]:
    config = networks[network_id]
    try:
        w3 = Web3(Web3.HTTPProvider(config['rpc']))
        usdc = w3.eth.contract(address=config['usdc'], abi=balance_abi)
        balance = usdc.functions.balanceOf(pay_to_address).call()
        balance_usdc = balance / 1e6
        
        # Markiere das erwartete Network
        marker = "← Erwartet" if network_id == CAIP2_NETWORK else ""
        # Markiere wenn Response ein anderes Network anzeigte
        if 'response_network' in dir() and response_network == network_id and network_id != CAIP2_NETWORK:
            marker = "← ⚠️ SETTLEMENT HIER!"
        
        print(f"   {config['name']} ({network_id}): ${balance_usdc:.2f} USDC {marker}")
    except Exception as e:
        print(f"   {config['name']}: Fehler - {e}")

print("\n📊 Base Networks:")
for network_id in ["eip155:8453", "eip155:84532"]:
    config = networks[network_id]
    try:
        w3 = Web3(Web3.HTTPProvider(config['rpc']))
        usdc = w3.eth.contract(address=config['usdc'], abi=balance_abi)
        balance = usdc.functions.balanceOf(pay_to_address).call()
        balance_usdc = balance / 1e6
        
        # Markiere das erwartete Network
        marker = "← Erwartet" if network_id == CAIP2_NETWORK else ""
        # Markiere wenn Response ein anderes Network anzeigte
        if 'response_network' in dir() and response_network == network_id and network_id != CAIP2_NETWORK:
            marker = "← ⚠️ SETTLEMENT HIER!"
        
        print(f"   {config['name']} ({network_id}): ${balance_usdc:.2f} USDC {marker}")
    except Exception as e:
        print(f"   {config['name']}: Fehler - {e}")

print(f"\n💡 Tipp: Vergleiche die Balances vor und nach dem Settlement,")
print(f"   um zu verifizieren, dass das Geld auf dem richtigen Network ankam.")

🔍 Cross-Chain Balance Check für Recipient: 0xAAEBC1441323B8ad6Bdf6793A8428166b510239C
   Erwartetes Settlement-Network: eip155:84532

   Optimism Mainnet (eip155:10): $1.63 USDC 
   Optimism Sepolia (eip155:11155420): $1.48 USDC 

💡 Tipp: Vergleiche die Balances vor und nach dem Settlement,
   um zu verifizieren, dass das Geld auf dem richtigen Network ankam.


In [38]:
# Check USDC balances to verify the transfer actually happened
if tx_hash:
    print(f"💰 Checking USDC Balances on {NETWORK_NAME}...\n")
    
    # USDC ERC-20 ABI for balanceOf
    erc20_abi = [
        {
            "inputs": [{"name": "account", "type": "address"}],
            "name": "balanceOf",
            "outputs": [{"name": "", "type": "uint256"}],
            "stateMutability": "view",
            "type": "function"
        }
    ]
    
    # Connect to network
    if USE_MAINNET:
        rpc_url = 'https://mainnet.optimism.io'
    else:
        rpc_url = 'https://sepolia.optimism.io'
    
    w3 = Web3(Web3.HTTPProvider(rpc_url))
    usdc_contract = w3.eth.contract(address=USDC_ADDRESS, abi=erc20_abi)
    
    # Check balances
    payer_balance = usdc_contract.functions.balanceOf(from_address).call()
    recipient_balance = usdc_contract.functions.balanceOf(pay_to_address).call()
    facilitator_balance = usdc_contract.functions.balanceOf(facilitator_address).call()
    
    print(f"📊 USDC Balances:")
    print(f"   Payer ({from_address[:10]}...): ${payer_balance/1e6:.6f} USDC")
    print(f"   Recipient ({pay_to_address[:10]}...): ${recipient_balance/1e6:.6f} USDC")
    print(f"   Facilitator ({facilitator_address[:10]}...): ${facilitator_balance/1e6:.6f} USDC")
    
    print(f"\n💡 Warum erscheint die Transaktion nicht in MetaMask Activity?")
    print(f"   • TransferWithAuthorization wird vom Facilitator ausgeführt")
    print(f"   • MetaMask zeigt nur selbst gesendete Transaktionen")
    print(f"   • Die USDC wurden trotzdem korrekt transferiert!")
    print(f"   • Prüfe den Token-Balance in MetaMask (nicht die Activity)")
    print(f"\n🔍 So siehst du den Transfer:")
    print(f"   1. Öffne Etherscan: {explorer_url}")
    print(f"   2. Klicke auf 'Logs' Tab um die Transfer Events zu sehen")
    print(f"   3. Oder gehe zu Token Transfers auf der Payer/Recipient Adresse")
else:
    print(f"\n⚠️  No transaction hash available")

💰 Checking USDC Balances on Base Sepolia (Testnet)...

📊 USDC Balances:
   Payer (0x55317955...): $0.000000 USDC
   Recipient (0xAAEBC144...): $0.000000 USDC
   Facilitator (0x3F8d2Fb6...): $0.000000 USDC

💡 Warum erscheint die Transaktion nicht in MetaMask Activity?
   • TransferWithAuthorization wird vom Facilitator ausgeführt
   • MetaMask zeigt nur selbst gesendete Transaktionen
   • Die USDC wurden trotzdem korrekt transferiert!
   • Prüfe den Token-Balance in MetaMask (nicht die Activity)

🔍 So siehst du den Transfer:
   1. Öffne Etherscan: https://sepolia.basescan.org/tx/0x6379e34692892294bc8edc93159200b9232be913c2ab93fcd902e06fe3d25781
   2. Klicke auf 'Logs' Tab um die Transfer Events zu sehen
   3. Oder gehe zu Token Transfers auf der Payer/Recipient Adresse
